<h1>Projeto 1 - Ciência dos Dados</h1>

Nome: *Caio Emmanuel*

Nome: *Larissa*

<h3>Introdução</h3>

Para realizar este trabalho, a dupla escolhe como tema a ser pesquisado as Olimpíadas de 2020 que ocorrerão no Japão por este ser um tema atual devido à preparação em velocidade recordista que o país teve para receber o evento e agora mais ainda devido à doença viral Corona que ameaça a realização do mesmo.

Classificamos os *tweets* obtidos como 0 ou 1, sendo 0 *tweets* irrelevantes sobre o assunto e 1 o contrário.

Vale ressaltar que usamos o artifício matemático probabilístico conhecido como *método de Naive-Bayes* para fazer previsões mais precisas sobre uma base de dados de teste.

<h3>Critérios de classificação</h3>

Classificamos como relevante aqueles *tweets* que remetem a assuntos atuais sobre a Olimpíada, como os atletas convidados, possível cancelamento, etc.

E classificamos como irrelevante aqueles que não têm relação com o evento mas tem com algum homônimo (olimpíadas científicas, regionais, olimpíadas passadas,etc.), além daqueles que julgamos não acrescentar nada ao assunto. 

<h3>Carregando algumas bibliotecas e o dataset:</h3>

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/caio/Documents/cdados/cdados_proj1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
dados_treino = pd.read_excel('olympics.xlsx')

In [4]:
dados_treino.head()

,Treinamento,Classification
0,rt @saitomri: i wrote about baseball. really. ...,0
1,rt @ndidi_amakaa: @solaadio olu leave your mot...,0
2,@wintersdancer they don’t want to lose the oly...,1
3,exclusive: indonesia considers 2032 olympics b...,0
4,the tentacles of canceling the tokyo olympics ...,1


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

<h3>Realizando limpeza dos dados</h3>

In [5]:
import re 

# Função que remove pontuação
def cleanup(text):
    punctuation = '[\/!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

for i in range(500):
    # Removendo pontuação
    dados_treino = dados_treino.replace(dados_treino['Treinamento'][i], cleanup(dados_treino['Treinamento'][i]))
    
    # Removendo '\n'
    dados_treino['Treinamento'] = dados_treino['Treinamento'].str.replace('[\n]','')
    
    # Removendo 'rt'
    if dados_treino['Treinamento'][i][:2] == 'rt':
        dados_treino = dados_treino.replace(dados_treino['Treinamento'][i], dados_treino['Treinamento'][i][3:])
        
    # Deixando todas as letras minúsculas:
    dados_treino = dados_treino.replace(dados_treino['Treinamento'][i], dados_treino['Treinamento'][i].lower())

In [6]:
dados_treino

,Treinamento,Classification
0,@saitomri i wrote about baseball really httpst...,0
1,@ndidi_amakaa @solaadio olu leave your mother ...,0
2,@wintersdancer they don’t want to lose the oly...,1
3,exclusive indonesia considers 2032 olympics bi...,0
4,the tentacles of canceling the tokyo olympics ...,1
...,...,...
495,highspeed camera used by nazis to both accurat...,0
496,indonesia considers 2032 olympics bid for new ...,0
497,@gailwalden6 i would like to apply to be in th...,0
498,the year is 2024 🔮womens world no 1 is 👩______...,0


<h3>Separando o dataset em *tweets* relavantes e irrelevantes</h3>

In [7]:
# tweets relevantes
tweets_r = dados_treino.loc[(dados_treino.Classification == 1)]
tweets_r

,Treinamento,Classification
2,@wintersdancer they don’t want to lose the oly...,1
4,the tentacles of canceling the tokyo olympics ...,1
5,it is the olympic year and we as a team have p...,1
9,@imrahultrehan from practicing muay thai in he...,1
10,athletes aiming for gold medals or sports busi...,1
...,...,...
478,@skift olympics cancellation would spell heavy...,1
480,olympics torch lighting ceremony will be close...,1
485,a lot of people stand to lose if olympics are ...,1
487,tokyo 2020 has been dealt its first significan...,1


In [8]:
# tweets irrelevantes
tweets_ir = dados_treino.loc[(dados_treino.Classification == 0)]

Logo a probabilidade de um *tweet* ser relevante (P_r) e irrelevante (P_ir) são...

In [9]:
P_r = tweets_r.Treinamento.count()/dados_treino.Treinamento.count()
P_ir = tweets_ir.Treinamento.count()/dados_treino.Treinamento.count()
print('A frequência de tweets relevantes é: {0}' .format(P_r))
print('A frequência de tweets irrelevantes é: {0}' .format(P_ir))

A frequência de tweets relevantes é: 0.356
A frequência de tweets irrelevantes é: 0.644


<h3>Calculando frequências relativas dentro de cada grupo</h3>

In [10]:
# criando texto com todos os tweets relevantes
txt_r = ''
for t in tweets_r['Treinamento']:
    t = str(t)
    txt_r += t

# criando série com as palavras dos tweets relevantes
words_r = pd.Series(txt_r.split())
words_r

0        @wintersdancer
1                  they
2                 don’t
3                  want
4                    to
             ...       
3634            torture
3635                yes
3636                for
3637                him
3638    @kagutamuseveni
Length: 3639, dtype: object

In [11]:
words_r = words_r.value_counts(True)
words_r

the             0.061830
olympics        0.035999
to              0.028030
of              0.019236
for             0.018137
                  ...   
sars            0.000275
running         0.000275
shot            0.000275
universities    0.000275
eight           0.000275
Length: 1378, dtype: float64

In [12]:
words_r.sum()

1.0

In [13]:
# Do mesmo jeito para os tweets irrelevantes:
txt_ir = ''
for t in tweets_ir['Treinamento']:
    t = str(t)
    txt_ir += t
    
words_ir = pd.Series(txt_ir.split())
words_ir = words_ir.value_counts(True)
words_ir

the                0.050114
olympics           0.028047
to                 0.024060
and                0.017796
in                 0.017227
                     ...   
decontamination    0.000142
@sinicouk          0.000142
lyrics             0.000142
vacations          0.000142
1976               0.000142
Length: 2672, dtype: float64

Assuma agora que as palavras que aparecem nos dois datasets anteriores sejam todas as palavras do Universo no Twitter (tweeverse)

In [14]:
txt = txt_r + txt_ir
tweeverse_words = pd.Series(txt.split())
tweeverse_words_frequencie = tweeverse_words.value_counts(True)

In [15]:
tweeverse_words_frequencie

the         0.054112
olympics    0.030761
to          0.025415
for         0.017256
of          0.017068
              ...   
rues        0.000094
18          0.000094
save        0.000094
hsright     0.000094
1976        0.000094
Length: 3514, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

<h3>Realizando testes</h3>

In [16]:
dados_teste = pd.read_excel('teste.xlsx')
dados_teste

,Teste,Classification
0,fwit olympics https://t.co/72go6l7pkx,0
1,@jimodonnell2 it's certainly a supportable opi...,1
2,@kapil857 @kapil857 any link to know how point...,0
3,rt @media_sai: what a superb performance by @s...,0
4,rt @theeconomist: japan may have to cancel the...,1
...,...,...
95,study: tokyo 2020 cancellation could see japan...,1
96,japanese doctors report sharp falls in cases o...,0
97,"rt @jimmfelton: “virus is bad, might have to c...",0
98,rt @drjitendrasingh: eight indian boxers inclu...,1


Vamos realizar todo o processo de formatação e encontra P_r e P_ir dos dados de treino

In [17]:
for i in range(100):
    # Removendo pontuação
    dados_teste = dados_teste.replace(dados_teste['Teste'][i], cleanup(dados_teste['Teste'][i]))
    
    # Removendo '\n'
    dados_teste['Teste'] = dados_teste['Teste'].str.replace('[\n]','')
    
    # Removendo 'rt'
    if dados_teste['Teste'][i][:2] == 'rt':
        dados_teste = dados_teste.replace(dados_teste['Teste'][i], dados_teste['Teste'][i][3:])
        
    # Deixando todas as letras minúsculas:
    dados_teste = dados_teste.replace(dados_teste['Teste'][i], dados_teste['Teste'][i].lower())

In [18]:
dados_teste

,Teste,Classification
0,fwit olympics httpstco72go6l7pkx,0
1,@jimodonnell2 its certainly a supportable opin...,1
2,@kapil857 @kapil857 any link to know how point...,0
3,@media_sai what a superb performance by @simra...,0
4,@theeconomist japan may have to cancel the oly...,1
...,...,...
95,study tokyo 2020 cancellation could see japan’...,1
96,japanese doctors report sharp falls in cases o...,0
97,@jimmfelton “virus is bad might have to cancel...,0
98,@drjitendrasingh eight indian boxers including...,1


Para classificarmos os tweets de teste, iremos usar a seguinte desigualdade:<br>
$$P(R|tweet) > P(IR|tweet)$$
Onde estamos verificando se a probabilidade de ser relevante é maior que a de ser irrelevante.
Mas pela lei de Naïve-Bayes temos:<br>
$$P(R|tweet) = \frac{P(tweet|R)P(R)}{P(tweet)}$$
<br>
$$P(IR|tweet) = \frac{P(tweet|IR)P(IR)}{P(tweet)}$$<br>
Como o denominador é comum a ambos, iremos comparar apenas os numeradores. Para isso, começaremos calculando $P(R)$ e $P(IR)$

In [19]:
# de uma maneira bem simplista:
P_R = len(txt_r.split())/len(txt.split())
P_IR = len(txt_ir.split())/len(txt.split())
P_R

0.3412735627872081

In [20]:
P_IR

0.6587264372127919

In [21]:
# agora calcularemos o P(tweet|R) e P(tweet|IR)
# perceba que esses são apenas o produto das probabilidades das palavras contidas nos tweets
# dado que são relevantes ou irrelevantes
P_TdadoR, P_TdadoIR = P_R, P_IR

# salvaremos os resultados numa lista para comparar depois
lista_relevante, lista_irrelevante = [], []

for tweet in dados_teste.Teste:
    for word in tweet.split():
        if word not in words_r:
            # aqui estamos desconsiderando palavras que não estão nesse espaço amostral
            pass
        else:
            P_TdadoR *= words_r[word]
    lista_relevante.append(P_TdadoR)
    P_TdadoR = P_R
len(lista_relevante)

100

In [22]:
for tweet in dados_teste.Teste:
    for word in tweet.split():
        if word not in words_ir:
            # aqui estamos desconsiderando palavras que não estão nesse espaço amostral
            pass
        else:
            P_TdadoIR *= words_ir[word]
    lista_irrelevante.append(P_TdadoIR)
    P_TdadoIR = P_IR
len(lista_irrelevante)

100

In [23]:
# agora vamos criar uma nova coluna no dataframe com a classificação segundo o classificador
classificador_NB = []
for i in range(len(lista_irrelevante)):
    if lista_irrelevante[i] > lista_relevante[i]:
        classificador_NB.append(0)
    else:
        classificador_NB.append(1)
dados_teste['Classificação NB'] = classificador_NB
dados_teste

,Teste,Classification,Classificação NB
0,fwit olympics httpstco72go6l7pkx,0,0
1,@jimodonnell2 its certainly a supportable opin...,1,0
2,@kapil857 @kapil857 any link to know how point...,0,1
3,@media_sai what a superb performance by @simra...,0,1
4,@theeconomist japan may have to cancel the oly...,1,1
...,...,...,...
95,study tokyo 2020 cancellation could see japan’...,1,1
96,japanese doctors report sharp falls in cases o...,0,1
97,@jimmfelton “virus is bad might have to cancel...,0,0
98,@drjitendrasingh eight indian boxers including...,1,0


In [24]:
# calculando a acurácia:
acertos = 0
for i in range(100):
    if dados_teste['Classification'][i] == dados_teste['Classificação NB'][i]:
        acertos += 1
print('Tivemos um total de {0}% de acertos' .format(acertos))

Tivemos um total de 37% de acertos


In [25]:
crosstab = pd.crosstab(dados_teste['Classification'],dados_teste['Classificação NB'])
crosstab

Classificação NB,0,1
Classification,,
0,11,41
1,22,26


In [26]:
print('Os resultados foram:\n\t{0}% são verdadeiros negativos\n\t{1}% são falsos negativos\n\t{2}% são falsos positivos\n\t{3}% são verdadeiros positivos' .format(crosstab[0][0], crosstab[1][0], crosstab[0][1], crosstab[1][1]))

Os resultados foram:
	11% são verdadeiros negativos
	41% são falsos negativos
	22% são falsos positivos
	26% são verdadeiros positivos


___
### Concluindo

<p>Podemos perceber que o nosso modelo correspondeu com a classificação inicial em 37% dos casos, entretanto, esse resultado é influenciado por fatores que não foram contabilizados e que poderia ter influenciado esse número negativamente ou positivamente.

Nosso classificador, por exemplo, faz uma análise muito simples e quantitativa e não leva em conta fatores como sarcasmo, quem foram os usuários marcados em uma postagem ou o significado cultural de um emoji e o contexto em que o tweet foi postado. Além do viés daqueles que classificaram inicialmente os tweets.

Podemos ainda refletir sobre as implicações de usar um modelo como esse para outros temas. Alguém que use esse modelo para prever o sentimento de pessoas quanto a um candidato X ou Y nas eleições presidenciais em um país poderia manipular os resultados a favor de um deste, uma vez que ele próprio classificou os tweets usados para treinamento. Ou seja, o modelo de Naïve-Bayes não é recomendado para situações tão delicadas.<p>

___
## Aperfeiçoamento:

<h3> Explicação de porque não podemos usar o classificador para gerar mais amostras </h3>
<br>
Como dito na conclusão, nosso modelo é muito simples e só considera aspectos matemáticos para tomar sua decisão. Por isso ele não é recomendado para julgar assuntos que dependam de aspectos tão humanos, como sarcasmo. Além de ele estar cheio de viés de quem criou a classificação inicial. Usá-lo para gerar mais amostras de treinamento faria com que ele tivesse uma acurácia de 100% quando testado com estas, o que resultaria num caso de overfitting.

<h3> Proposição de outros cenários para Naïve-Bayes </h3>
<br>
Ainda na conclusão dissemos que esse algoritmo não é aconselhado para situações tão delicadas como eleições, já que as conclusões podem alterar dinâmicas políticas, econômicas e sociais.
Ele também não é recomendado para dar diagnosticos a pacientes internados sobre suas chances de sobrevivência.
Entretanto, podemos usar esse para procurar e filtrar na internet tweets, posts de facebook, notícias em sites ou qualquer outro texto que tenha relação com assuntos mais concretos e menos sensíveis como textos que falem sobre o novo Corona vírus ou textos que falem sobre Ciência de Dados, já que estes costumam conter um jargão e escolhas de palavras bem característico. Além de que classificar um texto sobre o Corona vírus erroneamente como sendo um texto sobre chinelos Havainas não teria repercussão tão grande na sociedade quanto dizer que o candidato X tem mais chances que o Y. Por isso estas situações podem fazer uso de Naïve-Bayes perfeitamente.

<h3> Sugestão de melhorias </h3>
<br>
Algumas possíveis melhorias a serem feitas são:
<h5>1. Conseguir mais tweets de treinamento e dividir a classificação inicial entre mais pessoas (de preferência com ciclos sociais e culturais diferentes)</h5>
Essa melhoria é eficaz não só porque teríamos uma base de dados para treino maior (e consequentemente mais palavras) mas também porque dilui o viés da classificação inicial entre pessoas que pensam diferente e tomam decisões diferentes;
<h5>2. Usar artifícios como cross-validation</h5>
Um conceito comum em machine learning que consiste em separar o dataframe de treino em dataframes menores, treinar o modelo sobre alguns destes e testar em outros e fazer isso um número grande de vezes antes de testar no dataframe de testes. Segue um link para como podemos fazer isso explicado por professores do Kaggle:


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**